In [ ]:
from traffic.core import Traffic
from traffic.data import navaids
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/merged/osn/takeoffs28_mass_complete.parquet"
)

In [ ]:
def aligned_navpoint(
    flight,
    angle_precision: int = 10,
    time_precision: str = "2T",
    min_time: str = "20s",
    min_distance: int = 10,
):
    """
    Provided with a flight, returns the flight if it is aligned on the specified
    navpoint within the specified angle precision, time precision, minimum time and
    minimum distance, otherwise returns nothing. Can be used with traffic pipe.
    """
    #
    if aligned := flight.aligned_on_navpoint(
        navaids["VEBIT"],
        angle_precision,
        time_precision,
        min_time,
        min_distance,
    ).next():
        flight.data["route"] = "VEBIT"
        return flight

    elif aligned := flight.aligned_on_navpoint(
        navaids["GERSA"],
        angle_precision,
        time_precision,
        min_time,
        min_distance,
    ).next():
        flight.data["route"] = "GERSA"
        return flight

    elif aligned := flight.aligned_on_navpoint(
        navaids["DEGES"],
        angle_precision,
        time_precision,
        min_time,
        min_distance,
    ).next():
        flight.data["route"] = "DEGES"
        return flight

    elif aligned := flight.aligned_on_navpoint(
        navaids.extent("Switzerland").get("ZUE"),
        angle_precision,
        time_precision,
        min_time,
        min_distance,
    ).next():
        flight.data["route"] = "ZUE"
        return flight

    else:
        return None


t = (
    t.iterate_lazy()
    .pipe(aligned_navpoint)
    .eval(desc="processing", max_workers=20)
)

In [ ]:
output_dir = "/mnt/beegfs/store/MIAR/merged/osn"
t.to_parquet(f"{output_dir}/takeoffs28_route.parquet")

In [ ]:
t

In [ ]:
category_colors = {
    "VEBIT": "#1f77b4",
    "GERSA": "#2ca02c",
    "DEGES": "#ff7f0e",
    "ZUE": "#d62728",
}

t.data["color"] = t.data["route"].map(category_colors)

In [ ]:
import plotly.graph_objects as go
import numpy as np

zue = navaids.extent("Switzerland").get("ZUE")
vebit = navaids["VEBIT"]
gersa = navaids["GERSA"]
degas = navaids["DEGES"]

fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        lat=[zue.latitude],
        lon=[zue.longitude],
        mode="markers",
        marker=dict(size=9),
        text="ZUE",
    )
)

for flight in t.sample(500):
    fig.add_trace(
        go.Scattermapbox(
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            mode="lines",
            line=dict(width=1, color=flight.data["color"].iloc[0]),
            opacity=0.5,
            hoverinfo="none",
            showlegend=False,
        )
    )

fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=10,
        center=dict(
            lat=zue.latitude,
            lon=zue.longitude,
        ),
    ),
)

fig.update_layout(
    width=1200,
    height=1000,
    margin=dict(l=50, r=0, t=40, b=40),
)

fig.show()